In [3]:
import os
from time import sleep
from osgeo import gdal
import numpy as np
from glob import glob
import time
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import xarray as xr
import pygrib

In [ ]:
data_ds = xr.open_dataset('download.grib', engine='cfgrib')
print(data_ds)

In [4]:
data_ds = pygrib.open('download.grib')

In [46]:
n_step = 24
u_wind_arr = []
v_wind_arr = []
sst_arr = []
press_arr = []
precip_arr = []

for ii,g in enumerate(data_ds):
    print(ii)
    if ii in [x*5 for x in range(n_step)]:
        u_wind_arr.append(g.values[np.newaxis,...])
    elif ii in [1+x*5 for x in range(n_step)]:
        v_wind_arr.append(g.values[np.newaxis,...])
    elif ii in [2+x*5 for x in range(n_step)]:
        sst_arr.append(g.values[np.newaxis,...])
    elif ii in [3+x*5 for x in range(n_step)]:
        press_arr.append(g.values[np.newaxis,...])
    elif ii in [4+x*5 for x in range(n_step)]:
        precip_arr.append(g.values[np.newaxis,...])
    else:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [47]:
stack_u_wind = np.vstack(u_wind_arr)
stack_v_wind = np.vstack(v_wind_arr)
stack_sst = np.vstack(sst_arr)
stack_press = np.vstack(press_arr)
stack_precip = np.vstack(precip_arr)

In [73]:
final_data = np.vstack([stack_u_wind[np.newaxis,...],stack_v_wind[np.newaxis,...],
            stack_sst[np.newaxis,...],stack_press[np.newaxis,...],stack_precip[np.newaxis,...]])
final_data = np.swapaxes(final_data,0,1)

In [74]:
final_data = final_data[:,:,:720,:720]

In [75]:
sub_step = 6
final_clips = np.ones((2,int(n_step/(2*sub_step)),2))*sub_step
final_clips[0,:,0] = np.arange(0,n_step,sub_step*2)
final_clips[1,:,0] = np.arange(sub_step,n_step,sub_step*2)

In [78]:
final_ds = {}
final_ds['input_raw_data'] = final_data
final_ds['dims'] = np.array([[5,720,720]]).astype(np.int32)
final_ds['clips'] = final_clips.astype(np.int32)

In [80]:
np.savez('era5_train.npz', **final_ds)
np.savez('era5_valid.npz', **final_ds)

In [9]:
train = np.load('moving-mnist-example/moving-mnist-test.npz')

In [10]:
for key in train.keys():
    print(key)

clips
dims
input_raw_data


In [44]:
train['input_raw_data'].shape

(60000, 1, 64, 64)

In [45]:
train['clips'].shape

(2, 3000, 2)

In [59]:
train['clips'][1,:,0]

array([   10,    30,    50, ..., 59950, 59970, 59990], dtype=int32)